In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data'
addendum = r'\Purify_25percent_theta'

folder_lt3 = folder_lt3+addendum
folder_lt4 = folder_lt4+addendum
### ssro calibrations
day = '20160716_' # for ssro calibration

ssro_calib_lt3 = day+'091002'
ssro_calib_lt4 = day+'113952'

Density_Matrix = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [1]:
### get all timestamps!

unshifted_days = ['20160808','20160807','20160722','20160721','20160720','20160808','20160807','20160722','20160724','20160725']
shifted_days = ['20160811','20160823','20160822']


all_lt3 , all_lt4 = [],[]
offsets,offsets_ch1 = [],[] # Hold offset to compensate for timing change for new APD
correction_time = 2772.5e3-2791.5e3
start_offset_ch1 = 1.8e3

for d in unshifted_days+shifted_days:
        tstamp_lt3,tstamp_lt4 = Density_Matrix.tstamps_for_both_setups(d,contains = 'purify')
        #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
        all_lt3.extend(tstamp_lt3)
        all_lt4.extend(tstamp_lt4)
        if d in shifted_days:
            offsets.extend(np.zeros(np.shape(tstamp_lt3)))
            offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4)))           
        else:
            offsets.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
            offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)

20160808
20160807
20160722
20160721
20160720
20160808
20160807
20160722
20160724
20160725
20160811
20160823
20160822


In [ ]:
### load the data and shift timings where necessary
Density_Matrix.load_raw_data(all_lt3,all_lt4)
Density_Matrix.correct_pq_times(offsets = offsets, offsets_ch1 = offsets_ch1)

In [ ]:
### filter parameters
photon_channel = 2 # 0 or 1; 2 means both HH detectors
st_start = 2772.5e3#2763e3
st_len       = 40e3 #50 ns
st_len_w2    = 40e3
ch1_offset = -start_offset_ch1
max_reps_w2 = 150

In [ ]:
### apply filters to data
Density_Matrix.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                       st_len = st_len,
                                                       st_len_w2 = st_len_w2
                                                       )
Density_Matrix.apply_sync_filter_w1_w2(verbose = True,max_w2 = max_reps_w2)
Density_Matrix.apply_is_purified_filter(signature = '11', verbose = True) # we should change how this filter behaves!
Density_Matrix.apply_CR_before_filter(verbose=False)
Density_Matrix.apply_CR_after_filter(verbose=False)
Density_Matrix.attach_state_filtered_syncs(verbose = False)

In [ ]:
### acquire density matrix
Density_Matrix.get_DM_correlations(verbose = True,apply_ROC = True)
Density_Matrix.reconstruct_DMs(verbose = True,max_likelihood = False)
# Density_Matrix.plot_dm()

In [23]:
np.matrix?

In [25]:
paulis = [np.matrix([[1,0],[0,1]],dtype=complex),np.matrix([[0,1],[1,0]],dtype=complex),np.matrix([[0,-1j],[1j,0]],dtype=complex),np.matrix([[1,0],[0,-1]],dtype=complex)]

In [30]:
np.kron(paulis[3],paulis[3])

matrix([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j],
        [ 0.+0.j, -0.+0.j, -0.+0.j,  1.-0.j]])